In [ ]:
from dotenv import load_dotenv
load_dotenv(override=True)

In [ ]:
from langchain_ollama import ChatOllama
llama_chat = ChatOllama(base_url="http//localhost:11434",
                        model="gpt-oss:20b",
                        temperature=0,
                        max_tokens=100)

#### Define PLaywright Web browser Tool

In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser
# Enable Async Operation in Notebook
import nest_asyncio
nest_asyncio.apply()

async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools_by_name = {tool.name: tool for tool in tools }
tools_by_name

#### Test Tool Invoke

In [ ]:
navigation_tool = tools_by_name["navigate_browser"]
get_elements_tool = tools_by_name["get_elements"]
await navigation_tool.ainvoke({"url": "http://eaapp.somee.com/Employee"})
await get_elements_tool.ainvoke({"selector": "td", "attributes": ["innerText"]})

#### Agent to invoke Playwright Browser Tool

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
# Manually pick the essential tools
essential_tools = [
    next(t for t in tools if t.name == "navigate_browser"),
    next(t for t in tools if t.name == "get_elements"),
    next(t for t in tools if t.name == "extract_text")
]
browser_agent = create_agent(model=llama_chat,
                             tools=essential_tools,
                             system_prompt="""
                             You are a helpful assistant that can answer question using the provided tools.
                             If the question involves web scraping, use appropriate tool to get the data.
                             RULES:
1. You MUST call only ONE tool at a time.
2. After a tool call, wait for the observation.
3. Do not guess selectors.
4. First, use 'navigate_browser' to go to the URL.
                             Always structure the final response in JSON format.
                             """, 
                             debug=True)
query = ("Extract the table data from http://eaapp.somee.com/Employee page."
         "and get name of first 10 employees")
human_message = HumanMessage(content=query)
result = await browser_agent.ainvoke({"messages": [human_message]})
print(result["messages"][-1].content)